In [3]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Import feature and tree visualization
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from matplotlib import style

import plotly.express as px #for visualization


In [44]:
# create dataframe with project's data set
df = pd.read_csv('HR.csv')

### Data Preprocesssing

In [45]:
# deleting clearly unuseful variables (employee count, employee number, over 18, Standard Hours)
df = df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', "StandardHours"], axis=1)

# drop rows that contain at least one empty cell
df.dropna(axis='rows', thresh=1)

# feature engineering
def binary_map(feature):
    return feature.map({'Yes':1, 'No':0})

# encoding Gender categorical data
df['Gender'] = df['Gender'].map({'Male':1, 'Female':0})

# encoding Attrition and Overtime
binary_list = ['Attrition', 'OverTime']
df[binary_list] = df[binary_list].apply(binary_map)

# one-hot encoding of features with more than two categories
df = pd.get_dummies(df, drop_first=True)

# visualize correlation between features
# which correlation method makes most sense for our data set?
corr = df.corr(method='pearson')
fig = px.imshow(corr,width=800, height=800)
fig.show()

# potentially drop highly correleated features
# explain why we will or will not drop the feature
cor_matrix = df.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
df = df.drop(columns=to_drop, axis=1)
print("Highly correlated feature(s) dropped: ", to_drop)

# GLM regression model to see which features are most important in relation to target variable
pass

# take the exponent of the variable coefficents to determine which features are most important for training the model
pass

# feature scaling
pass

Highly correlated feature:  ['MonthlyIncome']


## ML Model

In [ ]:
# create baseline model
pass

# feature selection 
pass

# use different models
pass

## Export Final Model